In [ ]:
!pip install monai

In [ ]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pydicom
import random
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy.ndimage import shift
import torch.optim as optim
from torch.amp import GradScaler, autocast # 속도 및 메모리 최적화
from monai.transforms import (
    Compose, LoadImaged, Spacingd, Orientationd, EnsureChannelFirstd,
    ScaleIntensityRanged, Resized, MapTransform, SelectItemsd, CopyItemsd, ConcatItemsd,
    DeleteItemsd, RandFlipd, RandAffined,
    RandGridDistortiond, RandGaussianNoised, RandAdjustContrastd, 
    RandGaussianSmoothd, Transposed, ToTensord
)
from monai.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from monai.networks.nets import resnet18
import torchmetrics # AUC 계산을 쉽게 해주는 라이브러리
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import pickle  #파일저장에
from tqdm import tqdm  # 학습 진행 상황 시각화를 위해 추가
from torch.optim import AdamW
from joblib import Parallel, delayed    # cpu 사용하기 해해
import timm


# 0. 설정 및 경로
BASE_DIR = '/kaggle/input/rsna-2023-abdominal-trauma-detection/'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 20
CLASS_NAME_LIST = ['bowel', 'extravasation', 'kidney', 'liver', 'spleen', 'any_injury']
LEARNING_RATE = 1e-4

MANUAL_MODEL_SAVE_PATH = '/kaggle/working/manual_ct_convnext_v1.pth'
MANUAL_HISTORY_SAVE_PATH = '/kaggle/working/manual_ct_convnext_v1.pkl'

MONAI_MODEL_SAVE_PATH = '/kaggle/working/monai_ct_convnext_v1.pth'
MONAI_HISTORY_SAVE_PATH = '/kaggle/working/monai_ct_convnext_v1.pkl'


IMAGE_TARGET = (64,128,128)
NUM_SLICES = 64

# 전처리된 데이터를 저장할 폴더
SAVE_DIR = '/kaggle/working/'
os.makedirs(SAVE_DIR, exist_ok=True)

# 파일 읽기
train_df = pd.read_csv(f'{BASE_DIR}train_2024.csv') # 파일명 확인 필요 (보통 train.csv)
tags_df = pd.read_parquet(f'{BASE_DIR}train_dicom_tags.parquet')

# 고유 폴더 경로 추출 및 환자 ID 연결
tags_df['series_path'] = tags_df['path'].str.split('/').str[:-1].str.join('/')
unique_series = tags_df[['PatientID', 'series_path']].drop_duplicates()

data_dicts = []
for idx, row in unique_series.iterrows():
    p_id = int(row['PatientID'])
    s_path = row['series_path']
    
    # 해당 환자의 라벨 정보 가져오기
    patient_labels = train_df[train_df['patient_id'] == p_id]
    if len(patient_labels) == 0: continue # 라벨 없는 경우 제외
    labels = patient_labels.iloc[0]
    
    data_dicts.append({
        "image": f"{BASE_DIR}{s_path}",
        "patient_id": p_id,

        # 2진 분류 (Healthy, Injury) -> [1, 0] 또는 [0, 1] 형태가 됨
        "bowel": labels[['bowel_healthy', 'bowel_injury']].values.astype("float32"),
        "extravasation": labels[['extravasation_healthy', 'extravasation_injury']].values.astype("float32"),
        
        # 3중 분류 (Healthy, Low, High) -> [1, 0, 0], [0, 1, 0], [0, 0, 1] 형태가 됨
        "liver": labels[['liver_healthy', 'liver_low', 'liver_high']].values.astype("float32"),
        "kidney": labels[['kidney_healthy', 'kidney_low', 'kidney_high']].values.astype("float32"),
        "spleen": labels[['spleen_healthy', 'spleen_low', 'spleen_high']].values.astype("float32"),

        # any_injury가 1이면 "어딘가 이상함", 0이면 "완전 건강"
        "any_injury": np.array([1 - labels['any_injury'], labels['any_injury']]).astype("float32")
        
    })

patient_ids = train_df['patient_id'].unique()
train_ids, val_ids = train_test_split(patient_ids, test_size=0.2, random_state=42)
train_files = [d for d in data_dicts if d['patient_id'] in train_ids] # data_dicts에 patient_id 키 추가 필요
val_files = [d for d in data_dicts if d['patient_id'] in val_ids]

print(f"준비된 데이터 수: {len(data_dicts)}")
print(f"디바이스: {DEVICE}")

In [ ]:
class ManualAugmentd(MapTransform):
    def __init__(self, keys, prob=0.5):
        super().__init__(keys)
        self.prob = prob

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if random.random() > self.prob:
                continue

            image = d[key]
            
            if not isinstance(image, torch.Tensor):
                image = torch.from_numpy(image).float()
            else:
                image = image.float()
            
            # 데이터 형태: (S, C, H, W) -> 예: (64, 3, 128, 128)
            S, C, H, W = image.shape
            device = image.device
            
            # 1. Random Flip (좌우/상하)
            if random.random() < 0.5:
                image = torch.flip(image, dims=[3]) # 가로(W) 반전
            if random.random() < 0.2:
                image = torch.flip(image, dims=[2]) # 세로(H) 반전

            # Random Affine (Rotation & Scale) - MONAI의 RandAffined 역할
            # 모든 슬라이스에 '동일한' 변환을 적용해야 장기가 뒤틀리지 않음
            if random.random() < 0.3:
                # 회전각 (약 -10 ~ 10도)
                angle = random.uniform(-0.15, 0.15) 
                # 스케일 (0.9 ~ 1.1)
                scale = random.uniform(0.9, 1.1)
                
                # 변환 행렬 생성
                cos_a = np.cos(angle)
                sin_a = np.sin(angle)
                
                # PyTorch용 2x3 Affine Matrix (Rotation + Scale)
                # [ [sc, -ss, tx], [ss, sc, ty] ]
                # Translation(tx, ty)까지 추가하여 RandAffined와 더 똑같이 만듦
                theta = torch.tensor([
                    [cos_a * scale, -sin_a * scale, 0],
                    [sin_a * scale,  cos_a * scale, 0]
                ], dtype=torch.float).unsqueeze(0).to(device)
                
                # Grid 생성 및 적용 (Bilinear Interpolation 사용으로 부드러움)
                grid = F.affine_grid(theta, size=(1, C, H, W), align_corners=False).to(device)
                
                # 모든 슬라이스에 동일한 grid 적용을 위해 루프 대신 view 활용 최적화
                # (S, C, H, W)를 (S, C, H, W)로 변환
                image = F.grid_sample(image, grid.repeat(S, 1, 1, 1), mode='bilinear', padding_mode='zeros', align_corners=False)

            # Random Intensity (Contrast & Brightness)
            if random.random() < 0.2:
                gamma = random.uniform(0.7, 1.3)
                image = torch.pow(image - image.min(), gamma) + image.min()

            # Random Noise
            if random.random() < 0.2:
                noise = torch.randn_like(image) * 0.02
                image = image + noise

            d[key] = image
        return d

# ---------------------------------------------------------
# 방식 1: 직접 구현 고도화 (Manual)
# ---------------------------------------------------------
class CustomCTPreprocessor:
    def __init__(self, dicom_dir, target_shape, output_spacing=(1.5, 1.5, 1.5)):
        self.target_shape = target_shape
        self.output_spacing = output_spacing # 물리적 mm 단위 통일 (성능 향상의 핵심)

        self.result = self.process(dicom_dir)

    def load_and_sort_dicom(self, dicom_dir):
        """DICOM 로드 및 물리적 위치(Z축) 기준 정렬"""
        files = [pydicom.dcmread(os.path.join(dicom_dir, f)) for f in os.listdir(dicom_dir)]
        # ImagePositionPatient의 3번째 값(Z)으로 정렬해야 해부학적 순서가 맞음
        files.sort(key=lambda x: float(x.ImagePositionPatient[2]))
        return files

    def get_hu_image(self, slices):
        """Raw Pixel을 물리적 밀도 단위(HU)로 변환"""
        image = np.stack([s.pixel_array for s in slices]).astype(np.float32)
        
        # 장비별 Rescale Slope/Intercept 적용
        slope = slices[0].RescaleSlope
        intercept = slices[0].RescaleIntercept
        image = image * slope + intercept
        return image

    def apply_multi_window(self, hu_image):
        """
        [성능 향상 팁] 3개의 서로 다른 윈도우를 RGB 채널처럼 사용
        - Channel 0: Soft Tissue (전반적 장기)
        - Channel 1: Liver/Spleen (고대비 장기 특화)
        - Channel 2: Bone/Air (경계선 강조)
        """
        def windowing(img, wl, ww):
            lower, upper = wl - ww//2, wl + ww//2
            img_clip = np.clip(img, lower, upper)
            return (img_clip - lower) / (upper - lower)

        # 임상적 근거 의한 값
        # ch0 (50, 400) - 복부 표준:
        # 복부 장기(간, 비장, 신장 등)의 평균 밀도가 보통 40~60 HU입니다. 그래서 중심을 50으로 잡습니다.
        # 주변의 지방(-50)부터 약간의 석회화(+200)까지 넓게 보기 위해 폭을 400으로 설정합니다.
        # ch1 (30, 150) - 간 특화(고대비):
        # 간 내부의 미세한 출혈이나 종양은 주변 조직과 밀도 차이가 아주 적습니다(약 10~20 HU 차이).
        # 이걸 잡아내려면 폭(WW)을 아주 좁게(150) 줄여서 대비를 극대화해야 합니다. 그래야 미세하게 어두운 부분이 확연히 드러납니다.
        # ch2 (100, 700) - 광범위/뼈/혈관:
        # 조영제가 들어간 혈관이나 뼈 근처의 출혈은 밀도가 높습니다.
        # 더 높은 수치(+100 이상)까지 포함하면서, 전체적인 윤곽을 잃지 않기 위해 범위를 아주 넓게(700) 잡은 것입니다.
        ch0 = windowing(hu_image, 50, 400)   # Standard Abdomen
        ch1 = windowing(hu_image, 30, 150)   # High Contrast Liver
        ch2 = windowing(hu_image, 100, 700)  # Wide Range (Bone/Fluid)
        
        return np.stack([ch0, ch1, ch2], axis=0) # (3, D, H, W)

    def resample_isotropic(self, image, slices):
        """
        병원마다 다른 슬라이스 두께를 1.5mm로 통일
        이 과정을 거쳐야 모델이 장기의 '진짜 크기'를 배움
        픽셀 1개가 실제 몸속에서 몇 mm인가? 맞추는 작업입니다. 
        """
        # 현재 Spacing (Thickness, PixelSpacing_X, PixelSpacing_Y)
        current_spacing = np.array([
            float(slices[0].SliceThickness),
            float(slices[0].PixelSpacing[0]),
            float(slices[0].PixelSpacing[1])
        ])
        
        resize_factor = current_spacing / self.output_spacing
        # ndimage.zoom으로 물리적 비율 보정 (채널별로 반복)
        new_channels = []
        for c in range(image.shape[0]):
            resampled = ndimage.zoom(image[c], resize_factor, order=1)
            new_channels.append(resampled)
        
        return np.stack(new_channels, axis=0)

    def final_resize_and_norm(self, image):
        """
        최종 크기 조정 및 Z-Score 정규화
        단순히 이미지를 가로, 세로, 높이 128개의 칸으로 강제로 늘리거나 줄이는 것
        """
        # 1. 모델 규격(64x128x128)으로 리사이즈
        factors = [
            1.0, # Channel은 고정
            self.target_shape[0] / image.shape[1],
            self.target_shape[1] / image.shape[2],
            self.target_shape[2] / image.shape[3]
        ]
        image = ndimage.zoom(image, factors, order=1)
        
        # 2. Z-Score 정규화: (x - mean) / std
        # 0~1 정규화보다 모델의 수렴 속도가 훨씬 빠름

        # 1e-8 더하는 유유
        # 만약 특정 슬라이스(image[c])의 모든 픽셀 값이 똑같다면(예: 전부 검은색 배경만 있는 경우),
        # 해당 이미지의 표준편차(std)는 0이되어 나눌 수 없게 됩니다.
        # 그래서 float32에서 1e-8 / float16 1e-5 정도가 가장 적당한 "아주 작은 수"를 더합니다.
        # 그냥 무작정 너무 작게 잡으면 0으로 인실 할 수 있음
        for c in range(image.shape[0]):
            image[c] = (image[c] - image[c].mean()) / (image[c].std() + 1e-8)
            
        return image
        
    def process(self, dicom_dir):
        """전체 파이프라인 실행"""
        slices = self.load_and_sort_dicom(dicom_dir)
        hu_img = self.get_hu_image(slices)
        multi_win = self.apply_multi_window(hu_img)
        resampled = self.resample_isotropic(multi_win, slices)
        final_img = self.final_resize_and_norm(resampled)
        return final_img.astype(np.float32) # 최종 출력: (3, 128, 128, 128)


# ---------------------------------------------------------
# 방식 2: MONAI (비교를 위해 단계를 Manual과 맞춤)
# ---------------------------------------------------------
def get_monai_expert_pipeline():
    return Compose([
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        Orientationd(keys=["image"], axcodes="RAS"),
        # 1. 물리적 해상도 통일 (Isotropic Resampling)
        Spacingd(keys=["image"], pixdim=(1.5, 1.5, 1.5), mode="bilinear"),
        
        # 2. 멀티 윈도우 채널 생성 (이미지를 3개로 복사)
        CopyItemsd(keys=["image"], times=3, names=["img_soft", "img_liver", "img_bone"]),
        
        # 3. 각 복사본에 서로 다른 윈도우 적용
        ScaleIntensityRanged(keys=["img_soft"], a_min=-150, a_max=250, b_min=0.0, b_max=1.0, clip=True),
        ScaleIntensityRanged(keys=["img_liver"], a_min=-50, a_max=100, b_min=0.0, b_max=1.0, clip=True),
        ScaleIntensityRanged(keys=["img_bone"], a_min=-100, a_max=600, b_min=0.0, b_max=1.0, clip=True),
        
        # 4. 3개 채널을 하나로 합침 (3, 64, 128, 128)
        ConcatItemsd(keys=["img_soft", "img_liver", "img_bone"], name="image"),
        DeleteItemsd(keys=["img_soft", "img_liver", "img_bone"]),

        # 5. 최종 크기 조정 및 배경 제거 효과
        Resized(keys=["image"], spatial_size=IMAGE_TARGET)
    ])


class Timm_Model(torch.nn.Module):
    def __init__(self, model_name='convnext_tiny'):
        super().__init__()
        # 특징 추출기 (ConvNeXt)
        # num_classes = 1000 (기본값): 모델의 최종 출력이 1,000개의 숫자(카테고리 점수)로 나옵니다.
        # num_classes = 0: 1,000개를 맞히는 마지막 층을 아예 없애버립니다. 대신, 그 바로 직전 단계인 **'이미지의 핵심 특징 정보(Feature Vector)'**를 그대로 출력합니다.
        self.backbone = timm.create_model(model_name, pretrained=True, num_classes=0)
        self.dim = self.backbone.num_features # Base 기준 1024

        # 어텐션 풀링: 128장 중 수상한 놈을 골라내는 '심사위원'
        self.attention_net = nn.Sequential(
            nn.Linear(self.dim, 256),
            nn.Tanh(),
            nn.Dropout(0.1), # 추가
            nn.Linear(256, 1)
        )
        
        # "이상 징후 탐지" 전용 헤드 (의심 모델 역할)
        self.suspicion_head = nn.Sequential(
            nn.Linear(self.dim, 256),  # 1024개를 256개의 핵심 의심 후보로 압축
            nn.LayerNorm(256),  # 학습을 안정적으로 만들어줌
            nn.ReLU(),            # 중요한 의심 신호만 통과시킴
            nn.Dropout(0.2),      # 과적합 방지 (너무 예민해지는 것 방지)
            nn.Linear(256, 2)     # 최종 경보 [정상, 이상]
        )

        # "정밀 병명 분류" 전용 헤드 (분류 모델 역할)
        # 장기별 결과 2 or 3개 도출
        self.organ_heads = nn.ModuleDict({
            'bowel': nn.Linear(self.dim, 2),
            'extravasation': nn.Linear(self.dim, 2),
            'liver': nn.Linear(self.dim, 3),
            'kidney': nn.Linear(self.dim, 3),
            'spleen': nn.Linear(self.dim, 3)
        })

    def forward(self, x):
        # 2.5D 방식으로 전체 슬라이스 훑기
        b, s, c, h, w = x.shape
        chunk_size = 8 # 한 번에 처리할 슬라이스 개수 (메모리에 따라 조절)
        all_features = []
        
        for i in range(0, s, chunk_size):
            # x_chunk: (Batch, 16, 3, 128, 128)
            x_chunk = x[:, i : i + chunk_size] 
            
            # 2D 연산을 위해 일시적으로 배치 차원으로 합침
            x_chunk = x_chunk.reshape(-1, c, h, w) # (Batch*16, 3, 128, 128)
            
            # 백본 통과 (이 순간 메모리 사용량이 chunk_size만큼으로 제한됨)
            feat_chunk = self.backbone(x_chunk) # (Batch*16, 1024)
            
            # 다시 슬라이스 차원 분리 후 리스트에 저장
            feat_chunk = feat_chunk.view(b, -1, self.dim) 
            all_features.append(feat_chunk)

        # 모든 특징 합치기
        features = torch.cat(all_features, dim=1) # (Batch, 64, 1024)
    
        # features = self.backbone(x) # (B*64, 1024)
        # features = features.view(b, s, -1) # (B, 64, 1024)

        # Attention Pooling으로 '이상 지점' 증폭
        # 각 슬라이스의 수상함 점수 계산
        att_scores = self.attention_net(features) # (B, 64, 1)
        
         # 점수를 0~1 사이 비중(가중치)으로 변환
        att_weights = F.softmax(att_scores, dim=1) # (B, S, 1)
        
        # 가중치를 곱해서 하나로 합침 (가장 수상한 슬라이스 정보가 증폭됨)
        combined = torch.sum(features * att_weights, dim=1) # (B, 1024)

        # 결과 도출
        out = {k: head(combined) for k, head in self.organ_heads.items()}
        out['any_injury'] = self.suspicion_head(combined)

        return out


def process_one_item(idx, item, total_count, mode, pipeline=None):

    step = max(1, total_count // 100)

    if idx % step == 0:
        percent = (idx / total_count) * 100
        # flush=True를 써야 백그라운드 로그에 즉시 기록됩니다.
        print(f"[{mode}] Progress: {percent:.0f}% 완료 ({idx}/{total_count})", flush=True)
        
    """
    한 명의 환자 데이터를 전처리하고 파일로 저장하는 핵심 함수
    """
    p_id = item['patient_id']
    s_id = item['image'].split('/')[-1]
    
    # 저장 경로 설정 (128 사이즈 구분을 위해 이름에 포함 가능)
    save_path = os.path.join(SAVE_DIR, f"{mode}_{p_id}_{s_id}.npz")
    
    # 1. 이미 파일이 있으면 전처리 생략하고 바로 리턴 (시간 절약)
    if os.path.exists(save_path):
        new_item = item.copy()
        new_item['image'] = save_path
        return new_item

    try:
        # 2. 방식에 따른 전처리 수행
        if mode == "manual":
            # Manual 방식: 직접 짠 CustomCTPreprocessor 호출
            # TARGET_SIZE는 전역 변수(예: (128, 128, 128))를 참조합니다.
            pre = CustomCTPreprocessor(item['image'], target_shape=IMAGE_TARGET)
            img = pre.result.astype(np.float16)
        else:
            # MONAI 방식: 전달받은 monai_pipeline 호출
            processed = pipeline(item)
            img = processed["image"].detach().cpu().numpy().astype(np.float16)
            
        # (Channel, Depth, H, W) -> (Slices, Channel, H, W)
        # 결과 형태: (64, 3, 128, 128)
        img = np.transpose(img, (1, 0, 2, 3))
        
        # 3. 압축률이 적어 의미 없다 판단하여 그냥 저장
        np.savez_compressed(save_path, img)
        
        # 4. 경로를 업데이트한 새 딕셔너리 반환
        new_item = item.copy()
        new_item['image'] = save_path
        return new_item

    except Exception as e:
        print(f"[{mode}] Error ID {p_id}: {e}")
        return None


def monai_train_pipeline():
    return Compose([
        LoadNpyTransformd(keys=["image"]),

        # MONAI 3D 연산을 위해 차원 변경 (C, S, H, W) -> (3, 64, 128, 128)
        # MONAI는 첫 번째 차원을 무조건 Channel로 간주합니다.
        Transposed(keys=["image"], indices=(1, 0, 2, 3)),

        # 공간적 변형 (Spatial)
        # spatial_axis: 0=S(Slices), 1=H, 2=W
        RandFlipd(keys=["image"], prob=0.5, spatial_axis=1), # 좌우 반전
        RandFlipd(keys=["image"], prob=0.5, spatial_axis=2), # 상하 반전
        
        RandAffined(
            keys=["image"],
            prob=0.2,
            # (S, H, W) 각 축에 대한 회전/스케일
            rotate_range=(0.1, 0.1, 0.1), 
            scale_range=(0.1, 0.1, 0.1),
            translate_range=(10, 10, 10),
            padding_mode="zeros",
            mode="bilinear"
        ),
        
        # 형태적 변형 (Grid Distortion)
        # 매우 강력하지만 연산량이 많음. T4 x2에서는 CPU 병목을 확인하며 사용할 것.
        RandGridDistortiond(
            keys=["image"],
            prob=0.2,
            num_cells=(4, 4, 4),
            distort_limit=(-0.05, 0.05), # 변형 강도 추가 (옵션)
            mode="bilinear"
        ),
        
        # 강도 및 노이즈 (Intensity)
        RandGaussianNoised(keys=["image"], prob=0.2, mean=0.0, std=0.05),
        RandAdjustContrastd(keys=["image"], prob=0.2, gamma=(0.7, 1.3)),
        RandGaussianSmoothd(keys=["image"], prob=0.1, sigma_x=(0.5, 1.0)),
        
        # 모델 입력을 위해 다시 원래 차원으로 복구 (S, C, H, W)
        # Timm_Model이 (Batch, Slices, C, H, W)를 기대하므로
        Transposed(keys=["image"], indices=(1, 0, 2, 3)),

        ToTensord(keys=["image"] + CLASS_NAME_LIST),
        
        SelectItemsd(keys=["image"] + CLASS_NAME_LIST)
    ])


def monai_val_pipeline():
    return Compose([
        LoadNpyTransformd(keys=["image"]),
        ToTensord(keys=["image"] + CLASS_NAME_LIST),
        SelectItemsd(keys=["image"] + CLASS_NAME_LIST)
    ])
    

class LoadNpyTransformd(MapTransform):
    def __call__(self, data):
        d = dict(data)
        # npz 파일 로드
        with np.load(d["image"]) as data_file:
            # 저장할 때 사용했던 키인 'img'를 사용해 데이터를 꺼냅니다.
            img = data_file['img']
            
        # numpy 배열을 torch 텐서로 변환합니다.
        d["image"] = torch.from_numpy(img).float()
        return d


def evaluate(model, loader, epoch, criterion):
    model.eval()
    val_epoch_loss = 0
    results = {k: {"preds": [], "trues": []} for k in CLASS_NAME_LIST}

    with torch.no_grad():
        val_loop = tqdm(loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Validation]", leave=False)
        for batch in val_loop:
            inputs = batch["image"].to(DEVICE)
            outputs = model(inputs)

            # 분류
            loss = 0
            for k in CLASS_NAME_LIST:
                raw_pred = torch.softmax(outputs[k], dim=1).detach().cpu()
                pred = raw_pred.as_tensor() if hasattr(raw_pred, "as_tensor") else raw_pred

                raw_true = batch[k].detach().cpu()
                if raw_true.dim() > 1:
                    true = torch.argmax(raw_true, dim=1)
                else:
                    true = raw_true.long()
                true = true.as_tensor() if hasattr(true, "as_tensor") else true
                
                results[k]["preds"].append(pred)
                results[k]["trues"].append(true)

                # Loss 계산용 정답값 처리
                target = batch[k].to(DEVICE).float() # long() 변환 대신 float()으로 명시적 변환
                loss += criterion(outputs[k], target)
                    

            val_epoch_loss += loss.item()
            val_loop.set_postfix(val_loss=loss.item())

    avg_val_loss = val_epoch_loss / len(loader)
    
    # AUC 계산
    auc_results = {}
    for k in CLASS_NAME_LIST:
        all_preds = torch.cat(results[k]["preds"])
        all_trues = torch.cat(results[k]["trues"])
        num_classes = all_preds.shape[1] 
        
        # AUC 계산기는 CPU에서 수행하는 것이 안전함
        auc_metric = torchmetrics.AUROC(task="multiclass", num_classes=num_classes)
        auc_results[k] = auc_metric(all_preds, all_trues).item()
    
    return auc_results, avg_val_loss


def train(train_files_preprocessed, val_files_preprocessed, 
          train_pipeline, val_pipeline,
          model_save_path, history_save_path):    
    
    # 비교하기
    # 구분	    	BCEWithLogitsLoss		      		CrossEntropyLoss
    # 풀네임		    Binary Cross Entropy with Logits		(Multiclass) Cross Entropy
    # 주요 목적		이진 분류 (Yes or No)			    	다중 분류 (A, B, C 중 하나)
    # 출력 노드 수	1개 (0~1 사이의 확률)			    	N개 (각 클래스별 점수)
    # 활성 함수		Sigmoid (내장됨)				    	Softmax (내장됨)
    # 타겟 라벨		0.0 또는 1.0 (Float)			     	0, 1, 2... 인덱스 (Long)
    # 특징		    각 타겟이 독립적임 (Multi-label 가능)	타겟 간 경쟁 관계 (합이 1이 됨)
    
    train_ds = Dataset(data=train_files_preprocessed, transform=train_pipeline)
    val_ds = Dataset(data=val_files_preprocessed, transform=val_pipeline)
    
    train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=4, pin_memory=(DEVICE.type == 'cuda') )
    val_loader = DataLoader(val_ds, batch_size=2, shuffle=True, num_workers=4, pin_memory=(DEVICE.type == 'cuda') )
    
    # 2. 모델, 손실함수, 옵티마이저
    model = Timm_Model(model_name='convnext_tiny').to(DEVICE)
    if torch.cuda.device_count() > 1:
        print("2개의 GPU를 사용합니다.")
        model = nn.DataParallel(model) # 모델을 복사하여 양쪽 GPU에 분산
        
    criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.05)
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    
    # 3. 스케줄러 설정 (CosineAnnealingLR 예시)
    # T_max: 보통 전체 에포크 수로 설정합니다.
    # scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)
    
    # (참고) 만약 안정성을 중시한다면 아래 스케줄러를 사용하세요.
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    scaler = GradScaler('cuda', enabled=(DEVICE.type == 'cuda')) # DEVICE가 object이므로 .type 추가 권장
    
    history = {
        "train_loss": [],
        "val_loss": [],
        "auc_avg_loss": [],  # 이전에 만든 AUC도 기록
        "auc_details": []
    }
    for epoch in range(NUM_EPOCHS):
        model.train()
        train_epoch_loss = 0
    
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]", leave=False)
        for batch in train_loop:
            inputs = batch["image"].to(DEVICE)
            optimizer.zero_grad()
            
            with autocast('cuda'):
                outputs = model(inputs)
                
                # Loss 계산 (초기화 중요)
                loss = 0
                for k in CLASS_NAME_LIST:
                    target = batch[k].to(DEVICE)
                    # CrossEntropy는 target이 float(확률)이면 그대로, int(인덱스)면 long으로 변환 필요
                    if target.dtype != torch.float32:
                        target = target.float()
                    
                    if k == 'any_injury':
                        loss += criterion(outputs[k], target) * 2.0
                    else:
                        loss += criterion(outputs[k], target)
                
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
    
            train_epoch_loss += loss.item()
            train_loop.set_postfix(loss=loss.item())
    
        avg_train_loss = train_epoch_loss / len(train_loader)
    
    
        # 에포크 종료 후 성능 출력
        auc_results, avg_val_loss = evaluate(model, val_loader, epoch, criterion) # 실무에선 val_loader 사용 권장
        mean_auc = sum(auc_results.values()) / len(auc_results)
        
        history["train_loss"].append(avg_train_loss)
        history["val_loss"].append(avg_val_loss)
        history["auc_avg_loss"].append(mean_auc)
        history["auc_details"].append(auc_results)
    
        current_lr = optimizer.param_groups[0]['lr']
        
        # CosineAnnealingLR 사용 시 (에포크 끝날 때마다 호출)
        # scheduler.step()
        
        # ReduceLROnPlateau 사용시
        scheduler.step(avg_val_loss) 
        
        print(f"\n>>> Epoch {epoch+1} Summary")
        print(f"LR: {current_lr:.6f} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        print(f"Mean AUC: {mean_auc:.4f}")
        for organ, val in auc_results.items():
            print(f" - {organ:15s}: {val:.4f}")
        print("-" * 50)

        # 5. 모델 가중치 저장
        torch.save(model.state_dict(), f'{epoch}_{model_save_path}')
        print(f"✅ 모델 가중치 저장 완료: {epoch}_{model_save_path}")
        
        # 6. 학습 히스토리 저장 (Pickle)
        with open(history_save_path, 'wb') as file:
            pickle.dump(history, f'{epoch}_{file}')
        print(f"✅ 학습 히스토리 저장 완료: {epoch}_{history_save_path}")

    return history

    
def show_history(history):
    plt.figure(figsize=(15, 6))

    # 1. Loss 그래프 (Training vs Validation)
    plt.subplot(1, 3, 1)
    plt.plot(history["train_loss"], label="Train Loss", marker='o')
    plt.plot(history["val_loss"], label="Val Loss", marker='o')
    plt.title("Training & Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()
    
    # 2. Mean AUC 그래프
    # 키 이름을 val_auc_mean으로 수정했습니다.
    plt.subplot(1, 3, 2)
    plt.plot(history["auc_avg_loss"], label="Mean Val AUC", color='orange', marker='s')
    plt.title("Mean Validation AUC")
    plt.xlabel("Epoch")
    plt.ylabel("AUC")
    plt.grid(True)
    plt.legend()
    
    # 장기별로 리스트를 추출하여 그래프 그리기
    plt.subplot(1, 3, 3) # 1행 3열 중 3번째 (에러 해결 지점)
    for organ in CLASS_NAME_LIST:
        # 각 장기별 데이터를 추출하여 루프 안에서 그립니다.
        organ_auc_history = [epoch_data[organ] for epoch_data in history["auc_details"]]
        plt.plot(organ_auc_history, label=f"{organ}")
    
    plt.title("Validation AUC by Organ")
    plt.xlabel("Epoch")
    plt.ylabel("AUC")
    plt.ylim(0.4, 1.05) # AUC가 1일 수도 있으므로 1.05 정도로 설정
    plt.grid(True, linestyle='--')
    # 범례가 많을 수 있으므로 그래프 옆으로 뺍니다.
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small') 
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Manual 전처리
# print("Train 데이터 전처리 시작 (Parallel)...")
# # 1000개만 선택
# train_files_subset = train_files[2000:]
# total_len = len(train_files_subset) # 진행률 표시를 위해 1000으로 설정

# train_results = Parallel(n_jobs=-1)(
#     delayed(process_one_item)(idx, item, total_len, "manual") 
#     for idx, item in enumerate(train_files_subset)
# )
# # 에포크 에러 방지를 위해 None 제거 (주소록 업데이트)
# train_files_preprocessed = [r for r in train_results if r is not None]
# print("Train 데이터 전처리 종료 (Parallel)...")

# print("Val 데이터 전처리 시작 (Parallel)...")
# total_len = len(val_files)
# val_results = Parallel(n_jobs=-1)(
#     delayed(process_one_item)(idx, item, total_len, "manual") for idx,item in enumerate(val_files)
# )
# # 에포크 에러 방지를 위해 None 제거 (주소록 업데이트)
# val_files_preprocessed = [r for r in val_results if r is not None]
# print("Val 데이터 전처리 종료 (Parallel)...")

# # Manual 학습'
# manual_train_loader_pipeline = Compose([
#     LoadNpyTransformd(keys=["image"]),
#     ManualAugmentd(keys=["image"]), # 여기서 증강 실행!
# ])
# manual_val_loader_pipeline = Compose([
#     LoadNpyTransformd(keys=["image"]),
# ])
# print("=" * 25,"Manual","=" * 25)
# history = train(train_files_preprocessed, val_files_preprocessed, 
#                 manual_train_loader_pipeline, manual_val_loader_pipeline, 
#                 MANUAL_MODEL_SAVE_PATH, MANUAL_HISTORY_SAVE_PATH)

# show_history(history)

In [ ]:
# MONAI 전리리
pipeline = get_monai_expert_pipeline()

# 1000개만 선택
train_files_subset = train_files[:2000]
total_len = len(train_files_subset) # 진행률 표시를 위해 1000으로 설정

print("Train 데이터 전처리 시작 (Parallel)...")
train_results = Parallel(n_jobs=-1)(
    delayed(process_one_item)(idx, item, total_len, "monai", pipeline)
    for idx,item in enumerate(train_files_subset)
)
# 에포크 에러 방지를 위해 None 제거 (주소록 업데이트)
train_files_preprocessed = [r for r in train_results if r is not None]
print("Train 데이터 전처리 종료 (Parallel)...")

# print("Val 데이터 전처리 시작 (Parallel)...")
# total_len = len(val_files) # 진행률 표시를 위해 1000으로 설정
# val_results = Parallel(n_jobs=-1)(
#     delayed(process_one_item)(idx, item, total_len, "monai", pipeline)
#     for idx,item in enumerate(val_files)
# )
# # 에포크 에러 방지를 위해 None 제거 (주소록 업데이트)
# val_files_preprocessed = [r for r in val_results if r is not None]
# print("Val 데이터 전처리 종료 (Parallel)...")

# # MONAI 학습
# monai_train_loader_pipeline = monai_train_pipeline()
# monai_val_loader_pipeline = monai_val_pipeline()

# print("=" * 25,"Monai","=" * 25)
# history = train(train_files_preprocessed, val_files_preprocessed, 
#                 monai_train_loader_pipeline, monai_val_loader_pipeline,
#                 MONAI_MODEL_SAVE_PATH, MONAI_HISTORY_SAVE_PATH)

# show_history(history)